In [3]:
import nibabel as nib 
import numpy as np 
import matplotlib.pyplot as plt 
import nilearn.plotting as niplt
%matplotlib inline 

In [6]:
subs=['sub-145', 'sub-143', 'sub-016', 'sub-142', 'sub-141', 'sub-133', 'sub-140', 'sub-136', 
			'sub-084', 'sub-135', 'sub-137', 'sub-138', 'sub-111', 'sub-106', 'sub-134', 'sub-132', 'sub-144']

layer_names=['layer_'+str(i)+"_activations" for i in range(13)]
results_prefix="/scratch/sreejank/sl_results/slumlordreach/"
d='/jukebox/griffiths/bert-brains/'
data_dir=d+'slumlordreach_data/' 

In [11]:
anatomical_results=np.zeros((len(subs),len(layer_names)))
functional_results=np.zeros((len(subs),len(layer_names)))
big_mask=nib.load(data_dir+"whole_brain_mask.nii.gz").get_fdata().astype('bool')
for i,layer in enumerate(layer_names):
    for j,sub in enumerate(subs):
        f_result=nib.load(results_prefix+"encoding-"+layer+"/"+sub+"_whole_brain_functional_SL.nii.gz").get_fdata()[big_mask]
        functional_results[j,i]=f_result[f_result>=np.percentile(f_result.flatten(),99)].mean()
        a_result=nib.load(results_prefix+""+layer+"/"+sub+"_whole_brain_anatomical_SL.nii.gz").get_fdata()[big_mask]
        anatomical_results[j,i]=a_result[a_result>=np.percentile(a_result.flatten(),99)].mean()


In [19]:
def bootstrap_CI(data):
    sampling=[]
    for _ in range(1000):
        sampling.append(np.mean(np.random.choice(data,size=len(data),replace=True)))
    return np.percentile(sampling,(2.5,97.5))

In [21]:
diff=(functional_results-anatomical_results)/anatomical_results*100.0
[bootstrap_CI(diff[:,i]) for i in range(13)]

[array([-57.75311401, -26.48049364]),
 array([-82.60149276, -51.9252721 ]),
 array([-85.05328075, -43.01213175]),
 array([-79.56744228, -43.87010099]),
 array([-81.52604882, -48.58690295]),
 array([-86.81502291, -50.87601397]),
 array([-87.71421982, -56.58354873]),
 array([-86.27180796, -45.2257806 ]),
 array([-85.20288125, -34.93856526]),
 array([-83.46520354, -35.97053258]),
 array([-93.06976038, -39.43180313]),
 array([-97.63266149, -43.63462449]),
 array([-107.67085092,  -51.92136014])]

In [23]:
anatomical_results.mean()

0.029067122580497637